In [ ]:
!pip install greykite

In [ ]:
from collections import defaultdict
import warnings

warnings.filterwarnings("ignore")

import pandas as pd
import plotly

from greykite.common.data_loader import DataLoader
from greykite.framework.templates.autogen.forecast_config import ForecastConfig
from greykite.framework.templates.autogen.forecast_config import MetadataParam
from greykite.framework.templates.forecaster import Forecaster
from greykite.framework.templates.model_templates import ModelTemplateEnum
from greykite.framework.utils.result_summary import summarize_grid_search_results

In [ ]:
df = DataLoader().load_peyton_manning()
df.head()

,ts,y
0,2007-12-10,9.590761
1,2007-12-11,8.519590
2,2007-12-12,8.183677
3,2007-12-13,8.072467
4,2007-12-14,7.893572


In [ ]:
metadata = MetadataParam(
    time_col="ts",
    value_col="y",
    freq="D"
)

In [ ]:
# creates forecasts and stores the result
forecaster = Forecaster()  

# specify the config
fcst_congig = ForecastConfig(
    model_template=ModelTemplateEnum.SILVERKITE.name,
    forecast_horizon=365,
    coverage=0.95,
    metadata_param=metadata
)

# run the model
result = forecaster.run_forecast_config(
    df=df,
    config=fcst_congig
)

Fitting 3 folds for each of 1 candidates, totalling 3 fits


In [ ]:
cv_results = summarize_grid_search_results(
    grid_search=result.grid_search,
    decimals=2,
    cv_report_metrics=None, # concise output
    column_order=["rank", "mean_test", "split_test", "mean_train", "split_train", "mean_fit_time", "mean_score_time", "params"]
)

# transpose to save space in the printed output
cv_results["params"] = cv_results["params"].astype(str)
cv_results.set_index("params", drop=True, inplace=True)
cv_results.transpose()

params,[]
rank_test_MAPE,1
mean_test_MAPE,7.31
split_test_MAPE,"(5.01, 8.53, 8.39)"
mean_train_MAPE,4.2
split_train_MAPE,"(3.82, 4.25, 4.54)"
mean_fit_time,13.02
mean_score_time,1.13


In [ ]:
backtest = result.backtest
fig = backtest.plot()
plotly.io.show(fig)

In [ ]:
backtest_eval = defaultdict(list)
for metric, value in backtest.train_evaluation.items():
    backtest_eval[metric].append(value)
    backtest_eval[metric].append(backtest.test_evaluation[metric])
metrics = pd.DataFrame(backtest_eval, index=["train", "test"]).T
metrics

,train,test
CORR,0.754324,0.757055
R2,0.556393,-0.694746
MSE,0.317129,0.864868
RMSE,0.563142,0.929983
MAE,0.401152,0.856552
MedAE,0.301442,0.840986
MAPE,4.75639,11.3051
MedAPE,3.76305,11.2075
sMAPE,2.38659,5.31705
Q80,0.20089,0.186995


In [ ]:
forecast = result.forecast
fig = forecast.plot()
plotly.io.show(fig)

In [ ]:
fig = forecast.plot_components()
plotly.io.show(fig)

In [ ]:
summary = result.model[-1].summary()
print(summary)

================================ Model Summary =================================

Number of observations: 2964,   Number of features: 122
Method: Ridge regression
Number of nonzero features: 122
Regularization parameter: 148.5

Residuals:
         Min           1Q       Median           3Q          Max
      -2.344      -0.3605     -0.06506       0.2601        3.756

             Pred_col    Estimate  Std. Err Pr(>)_boot sig. code                   95%CI
            Intercept       8.047   0.01895     <2e-16       ***           (8.01, 8.083)
  events_C...New Year     0.01506   0.02483      0.546                (-0.03154, 0.0621)
  events_C...w Year-1     0.00384   0.02278      0.872                (-0.03819, 0.0516)
  events_C...w Year-2     0.01009   0.02329      0.684               (-0.03308, 0.05291)
  events_C...w Year+1     0.01914   0.02638      0.484                (-0.02862, 0.0702)
  events_C...w Year+2     0.02713    0.0263      0.310               (-0.02313, 0.08108)
 events

In [ ]:
# extract the model from the results
model = result.model

# create the future dataframe
future_df = result.timeseries.make_future_dataframe(
    periods=4,
    include_history=False)

# get predictions
model.predict(future_df)

,ts,forecast,forecast_lower,forecast_upper,y_quantile_summary,err_std
0,2016-01-21,8.964736,8.016267,9.913206,"(8.016266568612478, 9.913205705277191)",0.483922
1,2016-01-22,8.964897,7.924127,10.005667,"(7.9241270009730345, 10.005667160080467)",0.531015
2,2016-01-23,8.603602,7.559956,9.647248,"(7.559956355690183, 9.647248225095886)",0.532482
3,2016-01-24,9.083030,7.789760,10.376301,"(7.789759626997768, 10.376300700410866)",0.659844
